In [5]:
import pandapower as pp
import matplotlib.pyplot as plt
import numpy as np

# Crear una red de prueba
net = pp.create_empty_network()

# Crear barras (buses)
bus1 = pp.create_bus(net, vn_kv=500, name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=500, name="Bus 2")

# Crear línea de transmisión entre las dos barras
pp.create_line(net, bus1, bus2, length_km=500, std_type="679-AL1/86-ST1A 380.0")

# Potencia activa (P) en MW y reactiva (Q) en MVAr
S = 1200  # MVA
FP = 0.9
P = S * FP
Q = S * np.sqrt(1 - FP**2)

# Crear una carga en la barra 2 usando P y Q
load = pp.create_load(net, bus2, p_mw=P, q_mvar=Q, name="Load")

# Crear un generador en la barra 1 (barra slack)
pp.create_ext_grid(net, bus1, vm_pu=1.0, name="Grid Connection")

# Límites de tensión según la normativa chilena para alta tensión (500 kV)
upper_limit = 1.05  # 525 kV
lower_limit = 0.95  # 475 kV

# Simulación sin compensación shunt
load_changes = np.linspace(0.5 * P, 1.5 * P, 20)
voltages_bus1_no_shunt = []
voltages_bus2_no_shunt = []

for load_value in load_changes:
    net.load.loc[load, 'p_mw'] = load_value
    net.load.loc[load, 'q_mvar'] = Q * (load_value / P)
    pp.runpp(net, max_iteration=50, tolerance_mva=1e-3)
    voltages_bus1_no_shunt.append(net.res_bus.vm_pu[bus1])
    voltages_bus2_no_shunt.append(net.res_bus.vm_pu[bus2])

# Añadir compensación shunt para mantener tensiones dentro de los límites
shunt = pp.create_shunt(net, bus2, q_mvar=0.0, p_mw=0.0)

def ajustar_compensacion_shunt(load_value, q_mvar_base):
    net.load.loc[load, 'p_mw'] = load_value
    net.load.loc[load, 'q_mvar'] = Q * (load_value / P)
    pp.runpp(net, max_iteration=50, tolerance_mva=1e-3)
    q_mvar_ajuste = 0.0
    
    while net.res_bus.vm_pu[bus2] < lower_limit or net.res_bus.vm_pu[bus2] > upper_limit:
        if net.res_bus.vm_pu[bus2] < lower_limit:
            q_mvar_ajuste += 1.0
        elif net.res_bus.vm_pu[bus2] > upper_limit:
            q_mvar_ajuste -= 1.0
        
        net.shunt.loc[shunt, 'q_mvar'] = q_mvar_base + q_mvar_ajuste
        pp.runpp(net, max_iteration=50, tolerance_mva=1e-3)
    
    return net.shunt.loc[shunt, 'q_mvar']

# Simulación con compensación shunt
voltages_bus1_shunt = []
voltages_bus2_shunt = []
shunt_values = []

for load_value in load_changes:
    q_mvar_ajustado = ajustar_compensacion_shunt(load_value, 0.0)
    shunt_values.append(q_mvar_ajustado)
    voltages_bus1_shunt.append(net.res_bus.vm_pu[bus1])
    voltages_bus2_shunt.append(net.res_bus.vm_pu[bus2])

# Graficar los resultados
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(load_changes / P, voltages_bus1_no_shunt, marker='o', label='Bus 1 (sin shunt)')
plt.plot(load_changes / P, voltages_bus2_no_shunt, marker='o', label='Bus 2 (sin shunt)')
plt.axhline(y=upper_limit, color='r', linestyle='--', label='Límite superior (1.05 pu)')
plt.axhline(y=lower_limit, color='g', linestyle='--', label='Límite inferior (0.95 pu)')
plt.xlabel('Potencia Demandada (p.u.)')
plt.ylabel('Tensión (p.u.)')
plt.title('Tensión sin Compensación Shunt')
plt.legend()
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(load_changes / P, voltages_bus1_shunt, marker='o', label='Bus 1 (con shunt)')
plt.plot(load_changes / P, voltages_bus2_shunt, marker='o', label='Bus 2 (con shunt)')
plt.axhline(y=upper_limit, color='r', linestyle='--', label='Límite superior (1.05 pu)')
plt.axhline(y=lower_limit, color='g', linestyle='--', label='Límite inferior (0.95 pu)')
plt.xlabel('Potencia Demandada (p.u.)')
plt.ylabel('Tensión (p.u.)')
plt.title('Tensión con Compensación Shunt')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


LoadflowNotConverged: Power Flow nr did not converge after 50 iterations!